In [1]:
!pip install -q torchlibrosa


In [2]:
import os
import sys
import random
import numpy as np
import pandas as pd
import librosa
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm.notebook import tqdm

# Set seeds
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Config
CFG = {
    'sample_rate': 32000,
    'duration': 5,
    'n_mels': 128,
    'batch_size': 32,
    'num_workers': 4,
    'epochs': 100,
    'learning_rate': 1e-4,
    'in_chans': 3,  # ✅ 3 for ViT, 1 for HTSAT
}


In [3]:
train_df = pd.read_csv('/kaggle/input/birdclef-2025/train.csv')
taxonomy_df = pd.read_csv('/kaggle/input/birdclef-2025/taxonomy.csv')

# Full path
train_df['full_path'] = train_df['filename'].apply(
    lambda x: os.path.join('/kaggle/input/birdclef-2025/train_audio', x)
)

# Only existing files
train_df = train_df[train_df['full_path'].apply(os.path.exists)].reset_index(drop=True)
print(f"✅ Total files: {len(train_df)}")

# Encode labels
le = LabelEncoder()
train_df['label_idx'] = le.fit_transform(train_df['primary_label'])
CFG['num_classes'] = len(le.classes_)
print(f"✅ Encoded {CFG['num_classes']} classes")


✅ Total files: 28564
✅ Encoded 206 classes


In [4]:
train_df, val_df = train_test_split(
    train_df,
    test_size=0.2,
    stratify=train_df['primary_label'],
    random_state=42
)

print(f"✅ Train: {len(train_df)}, Val: {len(val_df)}")


✅ Train: 22851, Val: 5713


In [5]:
class BirdCLEFDataset(Dataset):
    def __init__(self, df, cfg, augment=False):
        self.df = df.reset_index(drop=True)
        self.cfg = cfg
        self.augment = augment

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        y, sr = librosa.load(row['full_path'], sr=self.cfg['sample_rate'])

        # Pad/truncate
        length = sr * self.cfg['duration']
        if len(y) < length:
            y = np.pad(y, (0, length - len(y)))
        else:
            y = y[:length]

        # Augmentation
        if self.augment:
            shift = int(random.uniform(-0.1, 0.1) * len(y))
            y = np.roll(y, shift)

        mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=self.cfg['n_mels'])
        mel_db = librosa.power_to_db(mel).astype(np.float32)

        # Normalize -1 to 1
        mel_db = (mel_db - mel_db.min()) / (mel_db.max() - mel_db.min())
        mel_db = 2 * mel_db - 1

        mel_db = torch.tensor(mel_db).unsqueeze(0)
        mel_db = F.interpolate(mel_db.unsqueeze(0), size=(224, 224), mode='bilinear', align_corners=False).squeeze(0)

        if self.cfg['in_chans'] == 3:
            mel_db = mel_db.repeat(3, 1, 1)

        return mel_db, row['label_idx']


In [6]:
train_ds = BirdCLEFDataset(train_df, CFG, augment=True)
val_ds = BirdCLEFDataset(val_df, CFG, augment=False)

train_dl = DataLoader(train_ds, batch_size=CFG['batch_size'],
                      shuffle=True, num_workers=CFG['num_workers'], pin_memory=True)
val_dl = DataLoader(val_ds, batch_size=CFG['batch_size'],
                    shuffle=False, num_workers=CFG['num_workers'], pin_memory=True)

# Sanity check
xb, yb = next(iter(train_dl))
print(f"✅ Batch shape: {xb.shape}, labels: {yb[:5]}")


✅ Batch shape: torch.Size([32, 3, 224, 224]), labels: tensor([ 75,  97, 179,  91, 144])


In [7]:
# Uncomment if using HTSAT
# sys.path.append('/kaggle/input/htsat-package-zip')
# from htsat.htsat import HTSAT_Swin_Transformer

# class DummyConfig:
#     sample_rate = 32000
#     window_size = 1024
#     hop_size = 320
#     mel_bins = 128
#     fmin = 50
#     fmax = 14000
#     enable_tscam = False

# model = HTSAT_Swin_Transformer(
#     spec_size=224,
#     patch_size=4,
#     in_chans=1,
#     num_classes=CFG['num_classes'],
#     window_size=7,
#     embed_dim=96,
#     depths=[2, 2, 6, 2],
#     num_heads=[3, 6, 12, 24],
#     ffn_expansion_factor=4,
#     qkv_bias=True,
#     config=DummyConfig()
# )

# ✅ OR use ViT
import timm
model = timm.create_model(
    "vit_base_patch16_224",
    pretrained=True,
    num_classes=CFG['num_classes'],
    in_chans=CFG['in_chans']
)

model = model.cuda()
print("✅ Model loaded")


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

✅ Model loaded


In [8]:
optimizer = optim.AdamW(model.parameters(), lr=CFG['learning_rate'])
criterion = nn.CrossEntropyLoss()

from torch.optim.lr_scheduler import ReduceLROnPlateau
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)

print("✅ Optimizer, loss, scheduler ready")


✅ Optimizer, loss, scheduler ready


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [9]:
debug_ds = Subset(train_ds, range(3))
for i in range(len(debug_ds)):
    mel_db, label = debug_ds[i]
    print(f"✅ Shape: {mel_db.shape}")
    print(f"✅ Min: {mel_db.min():.4f} Max: {mel_db.max():.4f}")
    print(f"✅ Mean: {mel_db.mean():.4f} Std: {mel_db.std():.4f}")
    print(f"✅ Label idx: {label}")


✅ Shape: torch.Size([3, 224, 224])
✅ Min: -1.0000 Max: 0.9843
✅ Mean: -0.3045 Std: 0.2426
✅ Label idx: 203
✅ Shape: torch.Size([3, 224, 224])
✅ Min: -1.0000 Max: 0.9845
✅ Mean: -0.0543 Std: 0.3728
✅ Label idx: 129
✅ Shape: torch.Size([3, 224, 224])
✅ Min: -1.0000 Max: 0.9584
✅ Mean: -0.1276 Std: 0.2466
✅ Label idx: 107


In [10]:
CHECKPOINT_DIR = "./checkpoints"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

early_stop_patience = 10
best_val_loss = float("inf")
epochs_no_improve = 0

print(f"✅ Checkpoint dir: {CHECKPOINT_DIR}")


✅ Checkpoint dir: ./checkpoints


In [11]:
# ✅ CELL 10 — Fixed-Epoch HTSAT Training Loop (40 Epochs) 🔒

import time
import torch.nn.functional as F
from torch.cuda.amp import GradScaler, autocast
from sklearn.metrics import accuracy_score

# Fixed config
EPOCHS = 40  # ⬅️ Fixed to 40 epochs
best_val_acc = 0.0

# Label smoothing loss
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

# Cosine learning rate scheduler
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

# Mixed precision scaler
scaler = GradScaler()

print(f"✅ Training for exactly {EPOCHS} epochs...")

for epoch in range(1, EPOCHS + 1):
    start = time.time()

    # --- TRAIN ---
    model.train()
    train_loss, correct, total = 0.0, 0, 0

    for xb, yb in tqdm(train_dl, desc=f"Train Ep{epoch}"):
        xb, yb = xb.cuda(), yb.cuda()

        optimizer.zero_grad()
        with autocast():
            logits = model(xb)
            loss = criterion(logits, yb)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        train_loss += loss.item() * xb.size(0)
        preds = logits.argmax(1)
        correct += (preds == yb).sum().item()
        total += yb.size(0)

    train_acc = correct / total
    train_loss /= total

    # --- VALIDATION ---
    model.eval()
    val_loss, val_correct, val_total = 0.0, 0, 0

    with torch.no_grad():
        for xb, yb in tqdm(val_dl, desc=f"Val Ep{epoch}"):
            xb, yb = xb.cuda(), yb.cuda()
            logits = model(xb)
            loss = criterion(logits, yb)

            val_loss += loss.item() * xb.size(0)
            preds = logits.argmax(1)
            val_correct += (preds == yb).sum().item()
            val_total += yb.size(0)

    val_acc = val_correct / val_total
    val_loss /= val_total

    scheduler.step()

    print(f"✅ Epoch [{epoch}/{EPOCHS}] "
          f"Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

    # Save checkpoints
    torch.save(model.state_dict(), f"{CHECKPOINT_DIR}/last.pth")
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), f"{CHECKPOINT_DIR}/best_model.pth")
        print("✅ Saved NEW BEST model")

    end = time.time()
    print(f"⏱️ Epoch time: {(end - start)/60:.2f} min")


✅ Training for exactly 40 epochs...


/tmp/ipykernel_19/1583097178.py:19: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


Train Ep1:   0%|          | 0/715 [00:00<?, ?it/s]

/tmp/ipykernel_19/1583097178.py:34: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Val Ep1:   0%|          | 0/179 [00:00<?, ?it/s]

...

Val Ep39:   0%|          | 0/179 [00:00<?, ?it/s]

✅ Epoch [39/40] Train Loss: 0.8572 Acc: 0.9991 | Val Loss: 1.9238 Acc: 0.7628
⏱️ Epoch time: 12.11 min


Train Ep40:   0%|          | 0/715 [00:00<?, ?it/s]

Val Ep40:   0%|          | 0/179 [00:00<?, ?it/s]

✅ Epoch [40/40] Train Loss: 0.8572 Acc: 0.9991 | Val Loss: 1.9237 Acc: 0.7630
⏱️ Epoch time: 12.00 min
